In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns=None
pd.options.display.max_colwidth=300
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_grad=pd.read_csv('/home/bea/Escritorio/KSchool_TFM/Principales/Encuesta_insercion_laboral_titulados_universitarios_2019/EILU_GRADUADOS/CSV/EILU_GRAD_2019.csv',
                    sep='\t')

/home/bea/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (119) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_grad.shape

(31651, 299)

In [4]:
for col in df_grad.columns:
    print(col,df_grad[col].dtype)

IDENT int64
SEXO int64
EDAD int64
NACIO int64
NACIO1 object
NACIO2 object
PAIS_NACI int64
TITU int64
RAMA int64
AMBITO int64
T_UNIV int64
DISCA int64
PROXY int64
LUG_RES_PAIS int64
PAIS_NAC_PADRE int64
ESTUDIOS_PADRE int64
PAIS_NAC_MADRE int64
ESTUDIOS_MADRE int64
EST_B1 int64
EST_B2_1 object
EST_B2_2 object
EST_B2_3 object
EST_B2_4 object
EST_B2_5 object
EST_M1 int64
EST_M2 object
EST_M3 object
EST_M4 object
EST_MES int64
EST_B8 int64
SAT1 int64
SAT2 object
EST_B11_1 int64
EST_B11_2 int64
EST_B11_3 int64
EST_B11_4 int64
EST_B11_5 int64
EST_B11_6 int64
EST_B11_7 int64
EST_B12 object
EST_B13_AMB1 object
EST_B14_1 object
EST_B15_1 object
EST_B13_AMB2 object
EST_B14_2 object
EST_B15_2 object
EST_B13_AMB3 object
EST_B14_3 object
EST_B15_3 object
EST_B16 object
EST_B17_M1 object
EST_B18_1 object
EST_B19_1 object
EST_B17_M2 object
EST_B18_2 object
EST_B19_2 object
EST_B17_M3 object
EST_B18_3 object
EST_B19_3 object
EST_B21 object
EST_B22 object
EST_B23 object
EST_B24 object
EST_B25_FA1 objec

#### 'IDENT'
**Índice** único del entrevistado.

In [5]:
df_grad['IDENT'].unique()

array([    1,     2,     3, ..., 31649, 31650, 31651])

#### 'SEXO'
**Sexo**: '1'=Hombre, '2'='Mujer'

In [6]:
df_grad['SEXO'].unique()

array([1, 2])

In [7]:
df_grad['SEXO'].value_counts()/df_grad['SEXO'].count()

2    0.570472
1    0.429528
Name: SEXO, dtype: float64

#### 'EDAD'
**Edad** del entrevistado: 1=Menores de 30 años, 2=De 30 a 34 años, 3=De 35 y más años. 

In [8]:
df_grad['EDAD'].unique()

array([1, 2, 3])

In [9]:
df_grad['EDAD'].value_counts()/df_grad['EDAD'].count()

1    0.496256
2    0.278822
3    0.224922
Name: EDAD, dtype: float64

#### 'NACIO'
**Nacionalidad** del entrevistado: 1=Española, 2=Española y otra, 3=Otra nacionalidad

In [10]:
df_grad['NACIO'].unique()

array([1, 2, 3])

In [11]:
df_grad['NACIO'].value_counts()/df_grad['NACIO'].count()

1    0.978137
3    0.011027
2    0.010837
Name: NACIO, dtype: float64

#### 'NACIO1'
**País de primera nacionalidad**: 1=Un país de la UE, 2=Un país fuera de la UE, 9=NS/NC.

In [12]:
df_grad['NACIO1'].unique()

array([' ', '2', '9', '1'], dtype=object)

In [13]:
df_grad[['NACIO','NACIO1']].groupby('NACIO').agg(['unique'])

,NACIO1
,unique
NACIO,
1,[ ]
2,"[2, 1, 9]"
3,"[9, 2, 1]"


In [14]:
# Se imputa nacionalidad de un país de la UE (1) a quien declaró nacionalidad española en variable 'NACIO'
df_grad['NACIO1'].mask(df_grad['NACIO']==1,'1',inplace=True)
df_grad['NACIO1']=df_grad['NACIO1'].astype('int')
df_grad[['NACIO','NACIO1']].groupby('NACIO').agg(['unique'])

,NACIO1
,unique
NACIO,
1,[1]
2,"[2, 1, 9]"
3,"[9, 2, 1]"


#### 'NACIO2'
**País de segunda nacionalidad**: 1=Un país de la UE, 2=Un país fuera de la UE, 9=NS/NC.

In [15]:
df_grad['NACIO2'].unique()

array([' ', '1', '2'], dtype=object)

In [16]:
df_grad[['NACIO','NACIO2']].groupby('NACIO').agg(['unique'])

,NACIO2
,unique
NACIO,
1,[ ]
2,[ ]
3,"[ , 1, 2]"


In [17]:
# Se imputa None a la no respuesta
df_grad['NACIO2'].replace({' ':None},inplace=True)
df_grad['NACIO2']=df_grad['NACIO2'].astype('float')
df_grad[['NACIO','NACIO2']].groupby('NACIO').agg(['unique'])

,NACIO2
,unique
NACIO,
1,[nan]
2,[nan]
3,"[nan, 1.0, 2.0]"


#### 'PAIS_NACI'
**País de nacimiento**: 1=España, 2=Otro país de la UE, 3=Otro país fuera de la UE, 9=NS/NC

In [18]:
df_grad['PAIS_NACI'].unique()

array([1, 3, 2])

#### 'TITU'
**Titulación**:  
011101=Pedagogía, 011201=Educación infantil, 011301=Educación primaria, 011401=Otros maestros, 011901=Educación social,   
021101=Audiovisual, imagen y multimedia, 021201=Diseño, 021301=Bellas artes, 021302=Historia del arte, 021401=Conservación y restauración, 021502=Música y Artes escénicas, 022201=Arqueología, 022202=Historia, 022301=Filosofía, 022901=Humanidades, 023101=Lengua inglesa, 023102=Lenguas clásicas, 023103=Otras lenguas extranjeras, 023104=Traducción e interpretación, 023201=Lenguas y dialectos españoles, 023202=Literatura, 023901=Lenguas modernas y aplicadas,   
031101=Economía, 031201=Política y gestión pública, 031202=Relaciones internacionales, 031301=Psicología, 031401=Antropología social y cultural y Estudios y gestión de la cultura, 031402=Criminología, 031404=Geografía, 031406=Sociología e Igualdad de género, 032101=Comunicación, 032102=Periodismo, 032201=Información y documentación,   
041201=Financiera y actuarial, 041202=Finanzas y contabilidad, 041301=Administración y empresa, 041302=Ciencias del trabajo, 041303=Gestión y administración pública, 041401=Marketing, 041402=Protocolo y eventos, 041403=Publicidad y relaciones públicas, 041601=Comercio, 042101=Derecho,   
051101=Biología, 051201=Bioquímica, 051202=Biotecnología, 051901=Biomedicina, 052101=Ciencias ambientales, 053101=Química, 053201=Ciencias del mar, 053202=Geografía y ordenación del territorio, 053203=Geología, 053301=Física, 054101=Matemáticas, 054201=Estadística,   
061301=Desarrollo de software y de aplicaciones e Ingeniería multimedia, 061901=Informática,   
071101=Ingeniería química industrial e Ingeniería medioambiental, 071301=Ingeniería de la energía, 071302=Ingeniería eléctrica, 071401=Ingeniería de computadores, 071402=Ingeniería de sonido e imagen, 071403=Ingeniería de telecomunicación, 071404=Ingeniería electrónica industrial y automática, 071405=Ingeniería en electrónica, 071501=Ingeniería en diseño industrial y desarrollo del producto, 071502=Ingeniería en tecnologías industriales, 071503=Ingeniería mecánica, 071601=Ingeniería aeronáutica, 071603=Ingeniería naval y oceánica, 071901=Ingeniería de organización industrial y Nanotecnología, 072101=Ciencia y tecnología de los alimentos e Ingeniería alimentaria, 072102=Enología, 072201=Ingeniería de materiales e Ingeniería textil, 072401	=Ingeniería de minas y energía, 073101=Arquitectura y Urbanismo y paisajismo, 073102=Ingeniería geomática, topografía y cartografía, 073201=Arquitectura técnica, 073202=Ingeniería civil,   
081102=Ingeniería agraria y agroalimentaria, 081103=Ingeniería agrícola, agropecuaria y medio rural, 081201=Ingeniería horticultura y jardinería, 082101=Ingeniería forestal y montes, 084101=Veterinaria,   
091101=Odontología, 091201=Medicina, 091301=Enfermería, 091401=Ingeniería biomédica y de la salud, 091402=Óptica y optometría, 091501=Fisioterapia, 091502=Logopedia, 091503=Nutrición humana y dietética, 091504=Podología, 091505=Terapia ocupacional, 091601=Farmacia, 092301=Trabajo social,   
101401=Actividad física y del deporte, 101501=Turismo, 104101=Náutica y transporte marítimo, 104103=Servicio de transporte terrestre y Servicio de transporte aéreo, 109999=Servicios (otros estudios).

In [19]:
df_grad['TITU'].unique()

array([ 11101,  11201,  11301,  11401,  11901,  21101,  21201,  21301,
        21302,  21401,  21502,  22201,  22202,  22301,  22901,  23101,
        23102,  23103,  23104,  23201,  23202,  23901,  31101,  31201,
        31202,  31301,  31401,  31402,  31404,  31406,  32101,  32102,
        32201,  41201,  41202,  41301,  41302,  41303,  41401,  41402,
        41403,  41601,  42101,  51101,  51201,  51202,  51901,  52101,
        53101,  53201,  53202,  53203,  53301,  54101,  54201,  61301,
        61901,  71101,  71301,  71302,  71401,  71402,  71403,  71404,
        71405,  71501,  71502,  71503,  71601,  71603,  71901,  72101,
        72102,  72201,  72401,  73101,  73102,  73201,  73202,  81102,
        81103,  81201,  82101,  84101,  91101,  91201,  91301,  91401,
        91402,  91501,  91502,  91503,  91504,  91505,  91601,  92301,
       101401, 101501, 104101, 104103, 109999])

#### 'RAMA'
**Rama de conocimiento** de la titulación:  
1=Artes y humanidades, 2=Ciencias, 3=Ciencias sociales y jurídicas, 4=Ingeniería y arquitectura, 5=Ciencias de la salud

In [20]:
df_grad['RAMA'].unique()

array([3, 1, 4, 5, 2])

In [21]:
df_grad['RAMA'].value_counts()/df_grad['RAMA'].count()

3    0.458248
4    0.211842
5    0.141796
1    0.100376
2    0.087738
Name: RAMA, dtype: float64

In [22]:
# Varias titulaciones adscritas a más de una rama del conocimiento
df_grad[['RAMA','TITU']].groupby('TITU').agg(['unique'])

,RAMA
,unique
TITU,
11101,"[3, 1]"
11201,[3]
11301,[3]
11401,[3]
11901,[3]
...,...
101401,"[3, 5]"
101501,[3]


In [23]:
df_grad[['RAMA','TITU']].groupby('RAMA').agg(['unique'])

,TITU
,unique
RAMA,
1,"[11101, 21101, 21201, 21301, 21302, 21401, 21502, 22201, 22202, 22301, 22901, 23101, 23102, 23103, 23104, 23201, 23202, 23901, 31401, 31404, 32101, 32102, 32201, 53202, 73101]"
2,"[51101, 51201, 51202, 51901, 52101, 53101, 53201, 53203, 53301, 54101, 54201, 71901, 72101, 72102, 91402, 91503]"
3,"[11101, 11201, 11301, 11401, 11901, 21101, 23104, 31101, 31201, 31202, 31301, 31401, 31402, 31404, 31406, 32101, 32102, 32201, 41201, 41202, 41301, 41302, 41303, 41401, 41402, 41403, 41601, 42101, 53202, 54201, 91501, 92301, 101401, 101501, 104103, 109999]"
4,"[21101, 21201, 53203, 54101, 61301, 61901, 71101, 71301, 71302, 71401, 71402, 71403, 71404, 71405, 71501, 71502, 71503, 71601, 71603, 71901, 72101, 72201, 72401, 73101, 73102, 73201, 73202, 81102, 81103, 81201, 82101, 91401, 104101, 104103, 109999]"
5,"[31301, 51101, 51901, 72101, 84101, 91101, 91201, 91301, 91402, 91501, 91502, 91503, 91504, 91505, 91601, 101401]"


#### 'AMBITO'
**Ámbito de estudio**: 0112=Formación de docentes de enseñanza infantil, 0113=Formación de docentes de enseñanza primaria, 011=Educación (Otros estudios), 0211=Técnicas audiovisuales y medios de comunicación, 021=Artes (Otros estudios), 022=Humanidades, 023=Lenguas, 0311=Economía, 0313=Psicología, 031=Ciencias sociales y del comportamiento (Otros estudios), 032=Periodismo y documentación, 0413=Dirección y administración, 041=Negocios y administración (Otros estudios), 042=Derecho, 051=Ciencias de la vida, 052=Medio ambiente, 053=Ciencias químicas, físicas y geológicas, 054=Matemáticas y estadística, 061=Informática, 071=Ingeniería y profesiones afines, 072=Industria manufacturera y producción, 073=Arquitectura y construcción, 081=Agricultura y ganadería, 082=Silvicultura, 084=Veterinaria, 0912=Medicina, 0913=Enfermería, 091=Salud (Otros estudios), 092=Servicios sociales, 1014=Actividades físicas y deportivas, 1015=Viajes, turismo y ocio, 109=Servicios (Otros estudios), 9999=NS/NC.

In [24]:
df_grad['AMBITO'].unique()

array([  11,  112,  113,  211,   21,   22,   23,  311,   31,  313,   32,
         41,  413,   42,   51,   52,   53,   54,   61,   71,   72,   73,
         81,   82,   84,   91,  912,  913,   92, 1014, 1015,  109])

In [25]:
# Todas las titulaciones adscritas a un único ámbito de estudio 
for row in range(0,len(df_grad[['TITU','AMBITO']].groupby('TITU'))):
    print (df_grad[['TITU','AMBITO']].groupby('TITU').agg(['unique']).iloc[row])

AMBITO  unique    [11]
Name: 11101, dtype: object
AMBITO  unique    [112]
Name: 11201, dtype: object
AMBITO  unique    [113]
Name: 11301, dtype: object
AMBITO  unique    [11]
Name: 11401, dtype: object
AMBITO  unique    [11]
Name: 11901, dtype: object
AMBITO  unique    [211]
Name: 21101, dtype: object
AMBITO  unique    [21]
Name: 21201, dtype: object
AMBITO  unique    [21]
Name: 21301, dtype: object
AMBITO  unique    [21]
Name: 21302, dtype: object
AMBITO  unique    [21]
Name: 21401, dtype: object
AMBITO  unique    [21]
Name: 21502, dtype: object
AMBITO  unique    [22]
Name: 22201, dtype: object
AMBITO  unique    [22]
Name: 22202, dtype: object
AMBITO  unique    [22]
Name: 22301, dtype: object
AMBITO  unique    [22]
Name: 22901, dtype: object
AMBITO  unique    [23]
Name: 23101, dtype: object
AMBITO  unique    [23]
Name: 23102, dtype: object
AMBITO  unique    [23]
Name: 23103, dtype: object
AMBITO  unique    [23]
Name: 23104, dtype: object
AMBITO  unique    [23]
Name: 23201, dtype: obje

In [26]:
# Varias ramas adscritas a más de un ámbito de estudio
df_grad[['RAMA','AMBITO']].groupby('AMBITO').agg(['unique'])

,RAMA
,unique
AMBITO,
11,"[3, 1]"
21,"[1, 4]"
22,[1]
23,"[1, 3]"
31,"[3, 1]"
32,"[3, 1]"
41,[3]
42,[3]


#### 'T_UNIV'
**Tipo de universidad**: 1=Universidad Pública presencial, 2=Universidad Pública a distancia, 3=Universidad Privada presencial, 4=Universidad Privada a distancia.

In [27]:
df_grad['T_UNIV'].unique()

array([2, 1, 4, 3])

In [28]:
df_grad['T_UNIV'].value_counts()

1    25898
3     3793
2     1080
4      880
Name: T_UNIV, dtype: int64

In [29]:
df_grad['T_UNIV'].value_counts()/df_grad['T_UNIV'].count()

1    0.818236
3    0.119838
2    0.034122
4    0.027803
Name: T_UNIV, dtype: float64

#### 'DISCA'
Tiene **discapacidad reconocida** superior al 33 %: 1=Sí, 2=No. 

In [30]:
df_grad['DISCA'].unique()

array([2, 1])

In [31]:
df_grad['DISCA'].value_counts()/df_grad['DISCA'].count()

2    0.987804
1    0.012196
Name: DISCA, dtype: float64

In [32]:
df_grad[['T_UNIV','DISCA']].groupby(['T_UNIV','DISCA']).agg({'DISCA':'count'})

DISCA
T_UNIV DISCA       
1      1        315
       2      25583
2      1         29
       2       1051
3      1         23
       2       3770
4      1         19
       2        861

In [33]:
df_grad[['T_UNIV','DISCA']].groupby('T_UNIV').agg({'DISCA':'count'})

,DISCA
T_UNIV,
1,25898
2,1080
3,3793
4,880


In [34]:
# Proporción de discapacidad en cada tipo de universidad: 
df_grad[['T_UNIV','DISCA']].groupby(['T_UNIV','DISCA']).agg({'DISCA':'count'}
                ).div(df_grad[['T_UNIV','DISCA']].groupby('T_UNIV').agg({'DISCA':'count'}),level='T_UNIV')

DISCA
T_UNIV DISCA          
1      1      0.012163
       2      0.987837
2      1      0.026852
       2      0.973148
3      1      0.006064
       2      0.993936
4      1      0.021591
       2      0.978409

In [35]:
# Proporción de universidad de cada tipo según discapacidad: 
df_grad[['DISCA','T_UNIV']].groupby(['DISCA','T_UNIV']).agg({'T_UNIV':'count'}
                ).div(df_grad[['DISCA','T_UNIV']].groupby('DISCA').agg({'T_UNIV':'count'}),level='DISCA')

T_UNIV
DISCA T_UNIV          
1     1       0.816062
      2       0.075130
      3       0.059585
      4       0.049223
2     1       0.818263
      2       0.033616
      3       0.120582
      4       0.027539